# 迁移学习

迁移学习是在一个任务上训练的模型被用于不同但相关的任务。在 PyTorch 中，进行迁移学习通常包括以下步骤：

- 加载预训练模型： 使用一个在大规模数据集上训练过的模型，例如 ImageNet 上的预训练模型。

- 修改模型架构： 修改加载的模型以适应新任务。这可能包括调整输出层的大小，冻结一些层，或者替换一些层。

- 定义新任务： 定义新任务的数据集和相应的损失函数。

- 训练模型： 使用新任务的数据集对修改后的模型进行训练。

In [1]:
import datetime as dt
import os
import warnings
from collections import UserDict
from tqdm import tqdm # 打印进度条

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 两种绘图接口
import plotly_express as px
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.nn.utils import weight_norm

%matplotlib inline

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(precision=2)

## 加载预训练模型

In [3]:
# 定义预训练模型
import os
import sys
path = 'D:\\Jupyter notebook\\src\\01.python\\04.实践应用\\01.时间序列\\models' #项目所在文件夹
sys.path.insert(0, path)

from lstm import LSTM
model_path = '../models/best_models/lstm.pth'
pretrained_model = torch.load(model_path)

ModuleNotFoundError: No module named 'lstm'